***GENERATED CODE FOR finaltest PIPELINE***

**CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM**

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(inStages, inStagesData, stageId, spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(inStages, inStagesData, stageId, spark, config):
        return inStagesData.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                                        delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


**TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA**

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_transform"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    indexed = indexed.drop(feature).withColumnRenamed(outcol, feature)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        feature).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            feature, dfReturn[feature].cast(IntegerType()))
        return changed_type_df
    # changed_type_df.show(3)
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': {'feature_label': 'sex'}, 'feature': 'sex', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '38', 'mean': '', 'stddev': '', 'min': 'female', 'max': 'male', 'missing': '0'}, 'transformation': 'String Indexer'})
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': {'feature_label': 'country'}, 'feature': 'country', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '38', 'mean': '', 'stddev': '', 'min': 'China', 'max': 'Korea', 'missing': '0'}, 'transformation': 'String Indexer'})
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': {'feature_label': 'region'}, 'feature': 'region', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '38', 'mean': '', 'stddev': '', 'min': 'Daegu', 'max': 'capital area', 'missing': '0'}, 'transformation': 'String Indexer'})
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': {'feature_label': 'infection_reason'}, 'feature': 'infection_reason', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '38', 'mean': '', 'stddev': '', 'min': 'contact with patient', 'max': 'visit to Wuhan', 'missing': '0'}, 'transformation': 'String Indexer'})
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': {'feature_label': 'state'}, 'feature': 'state', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '38', 'mean': '', 'stddev': '', 'min': 'deceased', 'max': 'released', 'missing': '0'}, 'transformation': 'String Indexer'})
        display(transformationDF.limit(2).toPandas())
        return transformationDF


**AUTOML FUNCTIONS**

In [ ]:
import json
from sklearn.cluster import KMeans
import optuna
import numpy as np
from sklearn import metrics
import pandas as pd


def kmeans(df, TrainingPercent):
    rows = df.count()
    percentrow = TrainingPercent / 100 * rows
    percentrows = int(percentrow)
    df = df.limit(percentrows)
    pd_df = df.toPandas()

    def objective(trial):
        n_clusters = int(trial.suggest_loguniform("n_clusters", 2, 101))
        classifier_obj = KMeans(n_clusters=n_clusters).fit(pd_df)
        labels = classifier_obj.predict(pd_df)
        score = metrics.silhouette_score(pd_df, labels, metric='euclidean')
        return score
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100)
    best = study.best_params
    best_param = int(best["n_clusters"])
    model = KMeans(n_clusters=best_param).fit(pd_df)
    labels = model.predict(pd_df)
    silhouette = metrics.silhouette_score(pd_df, labels, metric='euclidean')
    pd_df['prediction'] = pd.DataFrame(labels)
    labels_unique = np.unique(labels)
    totalClusters = len(labels_unique)
    display("totalClusters     : %s" % totalClusters)
    display(pd_df.head())
    return pd_df


class Clustering:

    def run(spark_DF, spark, config):
        stage_attribute = json.loads(config)
        stage_attribute['model']
        trainingPercent = stage_attribute['TrainingPercent']
        kmeans_model = kmeans(df=spark_DF, TrainingPercent=trainingPercent)


**READING DATAFRAME**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

try: 
	finaltestdbfs = DBFSConnector.fetch([], {}, "5ee9b69818d72208a8a23d0f", spark, "{'url': '/Demo/cluserting/coronaPatientNa.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")

except Exception as ex: 
	logging.error(ex)


**TRANSFORMING DATAFRAME**

In [ ]:
try: 
	finaltestautofe = TransformationMain.run(finaltestdbfs,json.dumps( {"FE": [{"transformationsData": {"feature_label": "sex"}, "feature": "sex", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "38", "mean": "", "stddev": "", "min": "female", "max": "male", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {}, "feature": "birth_year", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "38", "mean": "1975.5", "stddev": "16.44", "min": "1942", "max": "2000", "missing": "0"}}, {"transformationsData": {"feature_label": "country"}, "feature": "country", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "38", "mean": "", "stddev": "", "min": "China", "max": "Korea", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "region"}, "feature": "region", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "38", "mean": "", "stddev": "", "min": "Daegu", "max": "capital area", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "infection_reason"}, "feature": "infection_reason", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "38", "mean": "", "stddev": "", "min": "contact with patient", "max": "visit to Wuhan", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "state"}, "feature": "state", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "38", "mean": "", "stddev": "", "min": "deceased", "max": "released", "missing": "0"}, "transformation": "String Indexer"}, {"feature": "sex_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "38", "mean": "0.5", "stddev": "0.51", "min": "0", "max": "1", "missing": "0"}}, {"feature": "country_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "38", "mean": "0.03", "stddev": "0.16", "min": "0", "max": "1", "missing": "0"}}, {"feature": "region_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "38", "mean": "0.97", "stddev": "1.15", "min": "0.0", "max": "4.0", "missing": "0"}}, {"feature": "infection_reason_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "38", "mean": "0.55", "stddev": "0.95", "min": "0.0", "max": "4.0", "missing": "0"}}, {"feature": "state_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "38", "mean": "0.21", "stddev": "0.47", "min": "0", "max": "2", "missing": "0"}}]}))

except Exception as ex: 
	logging.error(ex)


**TRAIN MODEL**

In [ ]:
try: 
	finaltestclustering = Clustering.run(finaltestautofe,spark,json.dumps( {"autoClustering": 0, "orignalfile": "/Demo/cluserting/coronaPatientNa.csv", "distinct_column": "state", "model": "Kmeans", "TrainingPercent": 100, "run_id": "b9bb9cc4399d4e33a417aa0a37cc6dbf", "model_id": "5ee9bbc69ceb2046097d5180"}))

except Exception as ex: 
	logging.error(ex)
